In [ ]:
#Codes released by Huaxiang Song
#email: cn11028719@huas.edu.cn
#https://orcid.org/0000-0002-8235-0455

In [ ]:
#PLEASE ENSURE THAT THIS TECHNOLOGY IS NOT USED FOR MILITARY PURPOSES, 
#AND THE SOURCE CODE CAN BE REQUESTED FOR REASONABLE REASONS.

In [ ]:
%autosave 300

In [ ]:
import torch 
import torch.nn as nn 
import torch.optim as optim 
from torch.optim import lr_scheduler 
import torch.nn.functional as F
import numpy as np 

import torchvision 

from torchvision import datasets, models, transforms
from torch.cuda.amp import autocast, GradScaler 

import time 
import os 

In [ ]:
import detection.my_os_utilities as mytools

In [ ]:
weights_file_path = r'/home/jason/KerasPy/pretrained_weight'
project_path=r'..'# 

In [ ]:
model_weights_dict = {
    'efficientnet_b0':'efficientnet_b0_rwightman-3dd342df.pth',
    
    'efficientnet_b3':'efficientnet_b3_rwightman-cf984f9c.pth',  
    
}

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [ ]:
def get_recordfiles():
    global dataset_flag 
    dataset_flag = data_dir.split('/')[-2].split('_')[0]
    
    global TR_flag 
    TR_flag= 'TR'+ data_dir.split('/')[-1].split('-')[0] + data_dir[-1]
    
    global time_stamp 
    time_stamp = mytools.get_time_prefix('second')
    
    global dataset_log_str
    dataset_log_str = f'{dataset_flag}_{TR_flag}_{time_stamp}_{model_selection}'

    global training_logfile_name
    training_logfile_name = f'Log_{dataset_log_str}.txt'#
    
    global model_sav_pth_filename
    model_sav_pth_filename = f'{dataset_log_str}.pth'#

In [ ]:
#num_workers --------------------------
loader_workers = 2
#----------------------
pin_memory_bool=True
use_amp = True
#####optiizer_settings
BNorm_decay_setting = 0.
weight_decay_setting = 1e-08
eps_setting = 1e-08

In [ ]:
def get_setting_log():
    global settings_log_1
    settings_log_1 = '' 
    settings_log_1 += \
    f'sample_min_size ={sample_min_size}; sample_max_size ={sample_max_size};\n'
    settings_log_1 += \
    f'RHF_prob ={RHF_prob}; RVF_prob ={RVF_prob}; Rgray_prob ={Rgray_prob}; Rcontrast_prob ={Rcontrast_prob}; '
    settings_log_1 += f'; Cjit_prob ={Cjit_prob}; Gausblur_prob = {Gausblur_prob}\n'
    settings_log_1 += f'\nBNorm_decay_setting = {BNorm_decay_setting}; \n;'

In [ ]:
def my_get_detection_model_b0(num_classes, pretrained=True):
    global Roi_align_layers
    
    from torchvision.models.detection.rpn import AnchorGenerator
    from torchvision.models.detection import FasterRCNN
    from detection.backbone_utils import BackboneWithFPN

    backbone = models.efficientnet_b0(weights=None)

    if pretrained:
        weights = torch.load(os.path.join(weights_file_path, model_weights_dict['efficientnet_b0']))
    
        print(f'Faster-RCNN-b0-fpn {backbone.load_state_dict(weights)}')

    backbone = backbone.features

    #----------------------------------------------------------
    global return_layers,FPN_out_channels
    return_layers = {'2': '0', '3': '1','5':'2','7':'3'}
    
    in_channels_list =  [ 24, 40, 112, 320]
    
    
    backbone = BackboneWithFPN(backbone,return_layers, in_channels_list,FPN_out_channels)
    
    #####################
   
    roi_pooler = torchvision.ops.MultiScaleRoIAlign(
            featmap_names = Roi_align_layers,
            output_size=7,
            sampling_ratio=2
        )
    ###########
    global anchor_sizes
    
    aspect_ratios = ((0.5, 1.0, 2.0),) * len(anchor_sizes)
    anchor_generator = AnchorGenerator(anchor_sizes, aspect_ratios)
    #######################################################

    model = FasterRCNN(
            backbone=backbone,
            num_classes=num_classes,
            rpn_anchor_generator=anchor_generator,
            box_roi_pool=roi_pooler,
            min_size = sample_min_size, max_size = sample_max_size
        )  
    
    return model.to(device)

In [ ]:
def my_get_detection_model_b3(num_classes, pretrained=True):
    global Roi_align_layers
    
    from torchvision.models.detection.rpn import AnchorGenerator
    from torchvision.models.detection import FasterRCNN
    from detection.backbone_utils import BackboneWithFPN

    backbone = models.efficientnet_b3(weights=None)

    if pretrained:
        weights = torch.load(os.path.join(weights_file_path, model_weights_dict['efficientnet_b3']))
        print(f'Faster-RCNN-b3-fpn {backbone.load_state_dict(weights)}')

    backbone = backbone.features

    #----------------------------------------------------------
    global return_layers,FPN_out_channels
    return_layers = {'2': '0', '3': '1','5':'2','7':'3'}
    
    in_channels_list =  [ 32, 48, 136, 384]
    
    
    backbone = BackboneWithFPN(backbone,return_layers, in_channels_list,FPN_out_channels)
    
    #####################
   
    roi_pooler = torchvision.ops.MultiScaleRoIAlign(
            featmap_names = Roi_align_layers,
            output_size=7,
            sampling_ratio=2
        )
    ###########
    global anchor_sizes
    
    aspect_ratios = ((0.5, 1.0, 2.0),) * len(anchor_sizes)
    anchor_generator = AnchorGenerator(anchor_sizes, aspect_ratios)
    #######################################################

    model = FasterRCNN(
            backbone=backbone,
            num_classes=num_classes,
            rpn_anchor_generator=anchor_generator,
            box_roi_pool=roi_pooler,
            min_size = sample_min_size, max_size = sample_max_size
        )  
    
    return model.to(device)

In [ ]:
def get_transforms():
    from torchvision.transforms import v2

    ###################################################
    global my_transforms_train, sample_min_size,data_dir
    my_transforms_train = v2.Compose(
        [
            v2.ToImage(),
            ###
            v2.Resize(size=(sample_min_size,sample_min_size),antialias=True),
            
            v2.RandomApply(torch.nn.ModuleList([v2.ColorJitter(0.5,0.5,0.5),]),p = Cjit_prob),
            ###
            v2.RandomHorizontalFlip(p = RHF_prob), 
            ####
            v2.RandomVerticalFlip(p = RVF_prob),
            ####
            v2.RandomGrayscale(p = Rgray_prob),
            ###
            v2.RandomAutocontrast(p = Rcontrast_prob),
            ###
            v2.RandomApply(torch.nn.ModuleList([v2.GaussianBlur(kernel_size = 5),]),p = Gausblur_prob),
            ###
            v2.ToDtype(torch.float32, scale = False),
        ]
    )
    ######################################################3
    global my_transforms_val
    my_transforms_val = v2.Compose(
        [            
            v2.ToImage(), 
            v2.Resize(size=(sample_min_size,sample_min_size),antialias=True),
            v2.ToDtype(torch.float32, scale = False),
        ]
    )

In [ ]:
def get_datasets():
    get_transforms()
    ######################
    global image_datasets
    image_datasets = {}
    train_anno_json = f'train-annotation-{data_dir[-1]}.json'
    #################################################################################
    image_datasets['train'] = datasets.CocoDetection(root= os.path.join(data_dir, 'train'), 
                                annFile = os.path.join(data_dir, 'anno_dir',train_anno_json),
                                                     transforms = my_transforms_train,                                                 
                                        )

    #########################################################################################################
    
    if 'DIOR20' in data_dir or 'MAR20' in data_dir or 'ShipRS50' in data_dir:
        data_dir_val = os.path.dirname(data_dir)

        
    image_datasets['val'] = datasets.CocoDetection(root = os.path.join(data_dir_val, 'val'), 
                                annFile = os.path.join(data_dir, 'anno_dir','val-annotation.json'), \
                                                       transforms = my_transforms_val, 
                                                  )

In [ ]:
def wrap_datasets():
    global image_datasets
    image_datasets['train'] = datasets.wrap_dataset_for_transforms_v2(image_datasets['train'], \
            target_keys= ["boxes", 'labels','image_id'])
    
    image_datasets['val'] = datasets.wrap_dataset_for_transforms_v2(image_datasets['val'], \
                target_keys= ["boxes", "labels",'area','image_id','iscrowd'])


In [ ]:
def get_dataloaders():
    wrap_datasets()
    ##############
    global dataloaders
    dataloaders ={}
    dataloaders['train']= torch.utils.data.DataLoader(image_datasets['train'], batch_size = train_Bsize,
            shuffle=True, pin_memory=pin_memory_bool, persistent_workers = True,num_workers =loader_workers, \
                                                      collate_fn=lambda batch: tuple(zip(*batch)))
    dataloaders['val'] =  torch.utils.data.DataLoader(image_datasets['val'], batch_size = val_Bsize,\
            shuffle=True, pin_memory=pin_memory_bool, persistent_workers = True,num_workers=loader_workers,\
                                                      collate_fn=lambda batch: tuple(zip(*batch))) 

In [ ]:
def group_weight(module):
    group_decay = []
    group_no_decay = []
    for m in module.modules():
        if isinstance(m, nn.Linear):
            group_decay.append(m.weight)
            if m.bias is not None:
                group_no_decay.append(m.bias)
        elif isinstance(m, torch.nn.modules.conv._ConvNd):
            group_decay.append(m.weight)
            if m.bias is not None:
                group_no_decay.append(m.bias)
        elif isinstance(m, torch.nn.modules.batchnorm._BatchNorm):
            if m.bias is not None:
                group_no_decay.append(m.weight)
            if m.bias is not None:
                group_no_decay.append(m.bias)

    assert len(list(module.parameters())) == len(group_decay) + len(group_no_decay)
    groups = [dict(params=group_decay), dict(params=group_no_decay, weight_decay = BNorm_decay_setting)]
    return groups

In [ ]:
def get_optimizer():
    
    global model_ft,optimizer_ft,lr_init_Classify_model
    
    params_grouped = group_weight(model_ft)

    optimizer_ft = optim.AdamW(params_grouped,\
                               lr=lr_init_Classify_model,betas=(0.9,0.999),eps = eps_setting,\
                               weight_decay = weight_decay_setting)
    #############################################
    global exp_lr_scheduler
    exp_lr_scheduler = lr_scheduler.CosineAnnealingLR(optimizer_ft, 
                                T_max=T_max_setting, eta_min=eta_min_setting)

In [ ]:
def get_model_ft():
    global model_ft
    global num_classes,sample_min_size,sample_max_size,loader_workers,train_Bsize,val_Bsize,image_input_size
    if 'DIOR20' in data_dir:
        num_classes = 21 
        sample_min_size = image_input_size 
        sample_max_size = image_input_size

    elif 'MAR20' in data_dir:
        num_classes = 21 
        sample_min_size = image_input_size 
        sample_max_size = image_input_size
        
    elif 'ShipRS50' in data_dir:
        num_classes = 51 
        sample_min_size = image_input_size 
        sample_max_size = image_input_size
    #
    if model_selection == 'Faster-RCNN-b3-fpn':
        model_ft = my_get_detection_model_b3(num_classes)
        loader_workers = 3
    elif model_selection == 'Faster-RCNN-b0-fpn':
        model_ft = my_get_detection_model_b0(num_classes)
        loader_workers = 3   
            

In [ ]:
def init_for_training():
    global val_in_coco,logfile_path
    
    get_model_ft()
    
    get_datasets()

    get_dataloaders()
    
    #get_mosaic_dataloader()
    
    val_in_coco = convert_to_coco_api(dataloaders['val'].dataset)
    
    get_optimizer()

    get_recordfiles()

    get_setting_log()

    global logfile_path
    
    logfile_path = os.path.join(project_path,training_logfile_name)

    print('model is ready for training!!!')

In [ ]:
def my_training_accum():
    global val_in_coco,mosaic_prob,mosaic_switch
    global accum_step
    
    my_scaler = torch.cuda.amp.GradScaler(enabled = True)
    #
    with open(logfile_path,'a+') as record_file:
        record_file.write(str(time.ctime())+'\n')
        record_file.write(f'T_model_selection = {model_selection} \n study_target= {study_target} \n')
        record_file.write(f'training data_dir={data_dir} \n')
        record_file.write(f'train_Bsize = {train_Bsize*accum_step}; val_Bsize = {val_Bsize}; \n')
        record_file.write(f'train_Bsize_unaccum = {train_Bsize}; accum_step = {accum_step}; \n')
        record_file.write(f'Roi_align_layers = {Roi_align_layers}; anchor_sizes = {anchor_sizes}; \n')
        record_file.write(f'lr_init_Classify_model ={lr_init_Classify_model};  \n')
        record_file.write(f'lr_init_RCNN ={lr_init_RCNN};  \n')
        record_file.write(str(dataloaders['train'].dataset)+ '\n')
        record_file.write(str(dataloaders['val'].dataset)+ '\n')
        record_file.write(f'sample_min_size={sample_min_size}, sample_max_size={sample_min_size};\n')
        record_file.write( f'return_layers ={return_layers}; \n{para_settings}')
        record_file.write(f'T_max={T_max_setting} \n')
        record_file.write(f'mosaic_prob={mosaic_prob} \n')
        record_file.write(f'eta_min={eta_min_setting} \n\n')
        record_file.write('-' * 10+'DA Hyper-parameter setting'+'-' * 10+'\n')
        record_file.write(f'{settings_log_1} \n') 

        record_file.write('-' * 10+'start training'+'-' * 10+'\n')

    ###########################################
    since = time.time()
    best_acc = 0.0
    best_epoch = 0

    for epoch in range(train_epochs):
        epoch_since = time.time()

        mosaic_switch = True
            
        # training for one epoch
        log,mosaic_counter = train_one_epoch_accum(model_ft, optimizer_ft, dataloaders['train'], device, epoch, \
                              print_freq=100, scaler= my_scaler,mosaic_prob = mosaic_prob)

        # update the learning rate
        if ((epoch + 1)%accum_step) == 0:
            exp_lr_scheduler.step()

        with open(logfile_path,'a+') as record_file:
            training_time_elapsed = time.time() - epoch_since
            training_time_elapsed_str = \
             f'training complete in {training_time_elapsed//60:.0f}m {training_time_elapsed%60:.0f}s\n'
            record_file.write(f'epoch {epoch+1} of {train_epochs}:\n')
            record_file.write(f'{str(log)}\n{training_time_elapsed_str}')
            record_file.write('-' * 10+'\n')
            
        print(f'\nmosaic_times:{mosaic_counter}\n')
        torch.cuda.empty_cache()
    #----------------------------------------
        if epoch < epoch_skip_testing:
            continue

        
        # evaluate on the test dataset
        result = evaluate_resize(model_ft, dataloaders['val'], device=device)

        if result.coco_eval['bbox'].stats[1] > best_acc:
            best_acc = result.coco_eval['bbox'].stats[1]
            best_epoch = epoch
            #####
            torch.save(model_ft.state_dict(),os.path.join(project_path,model_sav_pth_filename))

        print('-' * 30+'\n')
        best_rec = f'best acc: {best_acc} in epoch {best_epoch +1 }\n'
        print(best_rec)
        
        epoch_time_elapsed = time.time() - epoch_since
        epoch_time_elapsed_str = \
             f'epoch complete in {epoch_time_elapsed//60:.0f}m {epoch_time_elapsed%60:.0f}s\n'

        with open(logfile_path,'a+') as record_file:  
            record_file.write('-' * 30+'\n')
            rec = result.coco_eval['bbox'].summarize_record
            record_file.write(f'{rec}\n')
            record_file.write(best_rec)
            record_file.write(epoch_time_elapsed_str)
            record_file.write('-' * 30+'\n')
    #end for---------------------------------------------------

    time_elapsed = time.time() - since

    with open(logfile_path,'a+') as record_file:
        record_file.write('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60)+'\n')
        record_file.write(f'best acc: {best_acc} in epoch {best_epoch +1 }\n')
    #### 
    print(('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60)+'\n'))
    print((f'best acc: {best_acc} in epoch {best_epoch +1 }\n'))

In [ ]:
def train_one_epoch_accum(model, optimizer, data_loader, device, epoch, print_freq, scaler=None,mosaic_prob=0.5):
    import math,gc,random
    from detection.utils import MetricLogger,SmoothedValue,reduce_dict
    from detection.coco_utils import get_coco_api_from_dataset
    
    global accum_step,mosaic_switch,using_two_mosaic,mosaic_combination 
    model.train()
    metric_logger = MetricLogger(delimiter="  ")
    metric_logger.add_meter("lr", SmoothedValue(window_size=1, fmt="{value:.6f}"))
    header = f"Epoch: [{epoch}]"
    mosaic_counter = 0
    
    batch_idx = 1 
    total_batches_num = len(data_loader)
    
    for images, targets in metric_logger.log_every(data_loader, print_freq, header):
        torch.cuda.empty_cache()
        
        images = list(image for image in images)
        targets = [{k: v if isinstance(v, torch.Tensor) else v for k, v in t.items()} for t in targets]
        
        #print(f'target length: {len(targets)}')
        if np.random.rand(1) < mosaic_prob and mosaic_switch:
            batchsize = len(images)
            resolution = images[0].shape[-1]
            
            mosaic_choice = random.sample(mosaic_combination,1)
            if mosaic_choice == [4]:
                if get_four_mosaic_image_and_boxes(images,targets,resolution=resolution,batchsize=batchsize) \
                  == True:
                    mosaic_counter+=1
            elif mosaic_choice == [9]:
                if get_nine_mosaic_image_and_boxes(images,targets,resolution=resolution,batchsize=batchsize) \
                  == True:
                    mosaic_counter+=1
            elif mosaic_choice == [3]:
                if get_three_mosaic_image_and_boxes(images,targets,resolution=resolution,batchsize=batchsize) \
                  == True:
                    mosaic_counter+=1
            elif mosaic_choice == [0]:
                if get_classic_mosaic_image_and_boxes(images,targets,resolution=resolution,batchsize=batchsize) \
                  == True:
                    mosaic_counter+=1
        
        images = [image.to(device) for image in images]
        targets = [{k: v.to(device) if isinstance(v, torch.Tensor) else v for k, v in t.items()} for t in targets]
        
        with torch.cuda.amp.autocast(enabled = scaler is not None):
            loss_dict = model(images, targets)
            losses = sum(loss for loss in loss_dict.values())
            losses = losses/(accum_step*1.0)
            
        del images
        

        # reduce losses over all GPUs for logging purposes
        loss_dict_reduced = reduce_dict(loss_dict)
        losses_reduced = sum(loss for loss in loss_dict_reduced.values())

        loss_value = losses_reduced.item()

        torch.cuda.empty_cache()
        #optimizer.zero_grad()
        #if scaler is not None:
        scaler.scale(losses).backward()
        if ((batch_idx % accum_step) == 0) or (batch_idx ==total_batches_num):  
            scaler.step(optimizer)
            scaler.update()
            optimizer.zero_grad()

        batch_idx += 1
   
        metric_logger.update(loss=losses_reduced, **loss_dict_reduced)
        metric_logger.update(lr=optimizer.param_groups[0]["lr"])
        
        del targets,loss_dict, losses
        torch.cuda.empty_cache()
    
    return metric_logger,mosaic_counter

In [ ]:
@torch.inference_mode()
def evaluate_resize(model, data_loader, device):
    from detection.engine import _get_iou_types
    from detection.coco_eval import CocoEvaluator
    from detection.utils import MetricLogger
    
    n_threads = torch.get_num_threads()
    # FIXME remove this and make paste_masks_in_image run on the GPU
    torch.set_num_threads(1)
    cpu_device = torch.device("cpu")
    model.eval()
    metric_logger = MetricLogger(delimiter="  ")
    header = "Test:"

    #coco = get_coco_api_from_dataset(data_loader.dataset)
    global val_in_coco
    coco = val_in_coco
    iou_types = _get_iou_types(model)
    coco_evaluator = CocoEvaluator(coco, iou_types)

    for images, targets in metric_logger.log_every(data_loader, 100, header):
        images = list(img.to(device) for img in images)
        
        '''for target in targets:
            for i in range(len(target['boxes'])):
                x1,y1,x2,y2 = target['boxes'][i]
                target['area'][i] = abs((x2-x1)*(y2-y1))'''  

        if torch.cuda.is_available():
            torch.cuda.synchronize()
        model_time = time.time()
        outputs = model(images)

        outputs = [{k: v.to(cpu_device) for k, v in t.items()} for t in outputs]
        model_time = time.time() - model_time

        res = {target["image_id"]: output for target, output in zip(targets, outputs)}
        evaluator_time = time.time()
        coco_evaluator.update(res)
        evaluator_time = time.time() - evaluator_time
        metric_logger.update(model_time=model_time, evaluator_time=evaluator_time)

    # gather the stats from all processes
    metric_logger.synchronize_between_processes()
    print("Averaged stats:", metric_logger)
    coco_evaluator.synchronize_between_processes()

    # accumulate predictions from all images
    coco_evaluator.accumulate()
    coco_evaluator.summarize()
    torch.set_num_threads(n_threads)
    return coco_evaluator

In [ ]:
def run_on_cycle_accum():
    
    global data_dir
    
    for i in range(len(data_dir_list)):        
        
        data_dir = data_dir_list[i]

        init_for_training()

        my_training_accum()

In [ ]:
def init_QDA_hyperparameters():
    global Cjit_prob,RHF_prob,RVF_prob,Rgray_prob,Rcontrast_prob,Gausblur_prob
    Cjit_prob = 0.
    RHF_prob = 0.
    RVF_prob = 0.
    Rgray_prob = 0.
    Rcontrast_prob = 0.
    Gausblur_prob = 0.

In [ ]:
def convert_to_coco_api(ds):
    from pycocotools.coco import COCO
    
    coco_ds = COCO()
    # annotation IDs need to start at 1, not 0, see torchvision issue #1530
    ann_id = 1
    dataset = {"images": [], "categories": [], "annotations": []}
    categories = set()
    for img_idx in range(len(ds)):
        # find better way to get target
        # targets = ds.get_annotations(img_idx)
        img, targets = ds[img_idx]       
                
        image_id = targets["image_id"]
        img_dict = {}
        img_dict["id"] = image_id
        img_dict["height"] = img.shape[-2]
        img_dict["width"] = img.shape[-1]
        dataset["images"].append(img_dict)
        bboxes = targets["boxes"].clone()
        bboxes[:, 2:] -= bboxes[:, :2]
        bboxes = bboxes.tolist()
        labels = targets["labels"].tolist()
        areas = targets["area"]#.tolist()
        if 'iscrowd' in targets:
            iscrowd = targets["iscrowd"]
        if "masks" in targets:
            masks = targets["masks"]
            # make masks Fortran contiguous for coco_mask
            masks = masks.permute(0, 2, 1).contiguous().permute(0, 2, 1)
        if "keypoints" in targets:
            keypoints = targets["keypoints"]
            keypoints = keypoints.reshape(keypoints.shape[0], -1).tolist()
        num_objs = len(bboxes)
        for i in range(num_objs):
            ann = {}
            ann["image_id"] = image_id
            ann["bbox"] = bboxes[i]
            ann["category_id"] = labels[i]
            categories.add(labels[i])
            #ann["area"] = areas[i]
            ###fix height & width
            box_height = bboxes[i][2]
            box_width = bboxes[i][3]
            ann["area"] = box_height * box_width
            ######            
            ann["iscrowd"] = iscrowd[i]
            ann["id"] = ann_id
            if "masks" in targets:
                ann["segmentation"] = coco_mask.encode(masks[i].numpy())
            if "keypoints" in targets:
                ann["keypoints"] = keypoints[i]
                ann["num_keypoints"] = sum(k != 0 for k in keypoints[i][2::3])
            dataset["annotations"].append(ann)
            ann_id += 1
    dataset["categories"] = [{"id": i} for i in sorted(categories)]
    coco_ds.dataset = dataset
    coco_ds.createIndex()
    return coco_ds

In [ ]:
def get_classic_mosaic_image_and_boxes(images,targets, resolution,batchsize=8,\
                                    minfrac=0.25, maxfrac=0.75,size_limit=2):
    import random
    from torchvision.tv_tensors._bounding_boxes import BoundingBoxes
    
    if batchsize<4: return batchsize
    
    s = resolution

    
    image_batch = []
    box_batch = []
    label_batch = []
    
    for ii in range(batchsize):
        #mosaic_size = s
        xc, yc = np.random.randint(resolution * minfrac, resolution * maxfrac, (2,)) 
        current_index = ii
        # random other 3 sample (could be the same too...)
        indice_list = [i for i in range(batchsize)]
        _ = indice_list.remove(current_index)
        indices = [current_index] + random.sample(indice_list, 3)

        mosaic_image = torch.zeros(3,s, s)
        final_boxes  = None
        final_labels = None
    
        for i, index in enumerate(indices):
            
            image = images[index].clone()
            
            boxes = targets[index]['boxes'].clone()
            
            labels = targets[index]['labels'].clone()
    

            if i == 0:    # top left
                x1o, y1o, x2o, y2o =  0,  0, xc, yc #coordinates in original image
                x1m, y1m, x2m, y2m =  0,  0, xc, yc #coordinates in mosaic image 
            elif i == 1:  # top right
                x1o, y1o, x2o, y2o =  xc,  0, s, yc #coordinates in original image
                x1m, y1m, x2m, y2m =  xc,  0, s, yc #coordinates in mosaic image 
            elif i == 2:  # bottom left
                x1o, y1o, x2o, y2o =  0,  yc, xc, s #coordinates in original image
                x1m, y1m, x2m, y2m =  0,  yc, xc, s #coordinates in mosaic image 
            elif i == 3:  # bottom right
                x1o, y1o, x2o, y2o =  xc,  yc, s, s #coordinates in original image
                x1m, y1m, x2m, y2m =  xc,  yc, s, s #coordinates in mosaic image 

            # cut image, save boxes
         
            mosaic_image[:,y1m:y2m, x1m:x2m] = image[:,y1o:y2o, x1o:x2o]
            if i == 0:
                #boxes
                final_boxes=boxes
                final_boxes[:, 0:3:2] = torch.clamp(final_boxes[:, 0:3:2], x1m, x2m)
                final_boxes[:, 1:4:2] = torch.clamp(final_boxes[:, 1:4:2], y1m, y2m)
                w = (final_boxes[:,2] - final_boxes[:,0])
                h = (final_boxes[:,3] - final_boxes[:,1])
                final_boxes = final_boxes[(w >= size_limit) & (h >= size_limit)]
                #label
                final_labels=labels
                final_labels = final_labels[(w >= size_limit) & (h >= size_limit)]
            elif i == 1:
                current_boxes=boxes
                current_boxes[:, 0:3:2] = torch.clamp(current_boxes[:, 0:3:2], x1m, x2m)
                current_boxes[:, 1:4:2] = torch.clamp(current_boxes[:, 1:4:2], y1m, y2m)
                w = (current_boxes[:,2] - current_boxes[:,0])
                h = (current_boxes[:,3] - current_boxes[:,1])
                current_boxes = current_boxes[(w >= size_limit) & (h >= size_limit)]
                #label
                current_labels=labels
                current_labels = current_labels[(w >= size_limit) & (h >= size_limit)]
                #concat
                final_boxes=torch.cat((final_boxes,current_boxes),0)
                final_labels=torch.cat((final_labels,current_labels),0)
            elif i == 2:
                current_boxes=boxes
                current_boxes[:, 0:3:2] = torch.clamp(current_boxes[:, 0:3:2], x1m, x2m)
                current_boxes[:, 1:4:2] = torch.clamp(current_boxes[:, 1:4:2], y1m, y2m)
                w = (current_boxes[:,2] - current_boxes[:,0])
                h = (current_boxes[:,3] - current_boxes[:,1])
                current_boxes = current_boxes[(w >= size_limit) & (h >= size_limit)]
                #label
                current_labels=labels
                current_labels = current_labels[(w >= size_limit) & (h >= size_limit)]
                #concat
                final_boxes=torch.cat((final_boxes,current_boxes),0)
                final_labels=torch.cat((final_labels,current_labels),0)
            elif i == 3:
                current_boxes=boxes
                current_boxes[:, 0:3:2] = torch.clamp(current_boxes[:, 0:3:2], x1m, x2m)
                current_boxes[:, 1:4:2] = torch.clamp(current_boxes[:, 1:4:2], y1m, y2m)
                w = (current_boxes[:,2] - current_boxes[:,0])
                h = (current_boxes[:,3] - current_boxes[:,1])
                current_boxes = current_boxes[(w >= size_limit) & (h >= size_limit)]
                #label
                current_labels=labels
                current_labels = current_labels[(w >= size_limit) & (h >= size_limit)]
                #concat
                final_boxes=torch.cat((final_boxes,current_boxes),0)
                final_labels=torch.cat((final_labels,current_labels),0)
                
        
        #break
        image_batch.append(mosaic_image)
        box_batch.append(final_boxes)
        label_batch.append(final_labels)
        
    #end loop
    images[:][:] = image_batch[:][:]
    for i in range(batchsize):
        targets[i]['boxes']=box_batch[i].to(device)
        targets[i]['labels']=label_batch[i].to(device)  
        
    del image_batch 
    del box_batch 
    del label_batch 
    
    global classic_mosaic_tag
    if classic_mosaic_tag == False:
        print('classic mosaic running... ')
        classic_mosaic_tag = True
        
    return True

In [ ]:
def init_mosaic_setting():
    global mosaic_combination,using_three_mosaic,using_four_mosaic, using_nine_mosaic,using_classic_mosaic
    mosaic_combination = []
    if using_three_mosaic: 
        mosaic_combination.append(3) 
    if using_four_mosaic: 
        mosaic_combination.append(4)
    if using_nine_mosaic: 
        mosaic_combination.append(9) 
    if using_classic_mosaic:
        mosaic_combination.append(0) 

    global three_mosaic_tag,four_mosaic_tag, nine_mosaic_tag,classic_mosaic_tag
    three_mosaic_tag = False
    four_mosaic_tag = False
    nine_mosaic_tag = False
    classic_mosaic_tag = False

    if len(mosaic_combination) == 0: 
        raise ValueError('mosaic_combination is empty')
    
    global para_settings
    para_settings = f'FPN_out_channels = {FPN_out_channels};\nusing_four_mosaic = {using_four_mosaic};\n'
    para_settings+= f'using_three_mosaic = {using_three_mosaic}\nusing_nine_mosaic = {using_nine_mosaic}\n'
    para_settings+= f'using_classic_mosaic = {using_classic_mosaic}\n'


In [ ]:
train_Bsize = 8 #Ensuring the codes work fine on your computer when your GPU memory is less than 16G
accum_step = 4

epoch_skip_testing = 0

train_epochs = 72 * 4

T_max_setting = 72    

lr_init_Classify_model = 1e-4

lr_init_RCNN = 1e-4

eta_min_setting = lr_init_Classify_model * 0.01

FPN_out_channels = 256

val_Bsize = 32

In [ ]:
data_dir_list = [r'/home/jason/data/MAR20_dataset/30%-Train-ratio-A',
                 
                ]

In [ ]:
model_selection = 'Faster-RCNN-b3-fpn'

study_target = 'GSConv-in-FPN_349_mosaic_LR=1E-4'

Roi_align_layers = ['0','1','2','3',]

anchor_sizes = ((24,), (48,), (96,), (144,), (192,))

using_resize = True

image_input_size = 640

mosaic_prob = 0.3

init_QDA_hyperparameters()

In [ ]:
epoch_skip_testing = 32

In [ ]:
using_classic_mosaic = True

In [ ]:
#PLEASE ENSURE THAT THIS TECHNOLOGY IS NOT USED FOR MILITARY PURPOSES, 
#AND THE SOURCE CODE CAN BE REQUESTED FOR REASONABLE REASONS.
using_four_mosaic = False

In [ ]:
#PLEASE ENSURE THAT THIS TECHNOLOGY IS NOT USED FOR MILITARY PURPOSES, 
#AND THE SOURCE CODE CAN BE REQUESTED FOR REASONABLE REASONS.
using_three_mosaic = False

using_nine_mosaic = False

In [ ]:
init_mosaic_setting()

In [ ]:
run_on_cycle_accum()

In [ ]:
%autosave 10

In [ ]:
os.system('shutdown +1')